# ***Libraries***

In [ ]:
import math
import os
import sys
import json

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

In [ ]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

In [ ]:
sys.path.append(parent_dir)

In [ ]:
from TSB_UAD.models.distance import Fourier
from TSB_UAD.models.feature import Window
from TSB_UAD.utils.slidingWindows import find_length, plotFig, printResult

from TSB_UAD.models.iforest import IForest

# ***Naive-Streaming Methods***
TODO: Add comments

## ***Data Pre-Processing***

In [ ]:
# Load the Time-Series dictionary
with open('Time-Series-Data-Dictionary.json', 'r') as json_file:
    loaded_dict = json.load(json_file)

In [ ]:
for filename, info in loaded_dict.items():
    print(f'{filename}: {info}')

In [ ]:
preprocessed_dict = {}

In [ ]:
from more_itertools import chunked

# Set the number of windows to be fit per batch.
windows_per_batch = 200

for filename, info in loaded_dict.items():
    ts_filepath = f"TS-Data-Files/{filename}"
    
    # === Pre-processing steps ===

    # Prepare data for unsupervised method
    ts = pd.read_csv(ts_filepath, header=None).dropna().to_numpy()

    name = ts_filepath.split('/')[-1]
    max_length = ts.shape[0]

    data = ts[:max_length, 0].astype(float)
    label = ts[:max_length, 1]

    slidingWindow = find_length(data)
    X_data = Window(window=slidingWindow).convert(data).to_numpy()

    # Take the series and batch it.
    batched_data = []
    i = 0
    while i < len(data):
        # print(len(data), i, i+slidingWindow+windows_per_batch, len(data[i:i+slidingWindow+windows_per_batch]))
        batched_data.append(data[i:i+slidingWindow+windows_per_batch - 1])
        i += windows_per_batch
        if i+slidingWindow+windows_per_batch > len(data):
            batched_data.append(data[i:min(i+slidingWindow+windows_per_batch - 1, len(data))])
            # print('end', len(data), i, i+slidingWindow+windows_per_batch, len(data[i:i+slidingWindow+windows_per_batch]))
            break
        # print(len(data), i, i+slidingWindow+windows_per_batch, len(data[i:i+slidingWindow+windows_per_batch]))

        
    # Take the windows and batch them.
    batched_X_data = [np.array(list(batch)) for batch in chunked(X_data, windows_per_batch)]

    # Prepare data for semisupervised method. 
    # Here, the training ratio = 0.1

    data_train = data[:int(0.1 * len(data))]
    data_test = data

    X_train = Window(window=slidingWindow).convert(data_train).to_numpy()
    X_test = Window(window=slidingWindow).convert(data_test).to_numpy()

    print(f'Time-Series filename: {filename}')
    print("Estimated Subsequence length: ", slidingWindow)
    print()
    
    # Store the pre-processed variables in the new dictionary
    preprocessed_dict[filename] = {
        'name': name,
        'data': data,
        'batched_data': batched_data,
        'label': label,
        'slidingWindow': slidingWindow,
        'X_data': X_data,
        'batched_X_data': batched_X_data,
        'data_train': data_train,
        'data_test': data_test,
        'X_train': X_train,
        'X_test': X_test,
        'Time series length': len(data),
        'Number of abnormal points': list(label).count(1)
    }

In [ ]:
265-125

In [ ]:
len(preprocessed_dict['ts1']['batched_X_data'][-1])

In [ ]:
len(preprocessed_dict['ts1']['batched_data'][-1])

140

: 

In [ ]:
65+65

In [ ]:
# Get filenames, time series lengths, and number of abnormal points
filenames = list(preprocessed_dict.keys())
time_series_lengths = [data['Time series length'] for data in preprocessed_dict.values()]
number_of_abnormal_points = [data['Number of abnormal points'] for data in preprocessed_dict.values()]

# Plot 'Time series length' and 'Number of abnormal points' for each filename
plt.figure(figsize=(10, 5))
plt.plot(filenames, time_series_lengths, marker='o', linestyle='-', color='skyblue')
plt.xlabel('Filename')
plt.ylabel('Time series length')
plt.title('Time Series Length for Each Filename')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(filenames, number_of_abnormal_points, marker='o', linestyle='-', color='lightgreen')
plt.xlabel('Filename')
plt.ylabel('Number of abnormal points')
plt.title('Number of Abnormal Points for Each Filename')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## ***Anomaly Detection***

### ***Isolation Forest***

In [ ]:
from tqdm.notebook import tqdm_notebook as tqdm

modelName = 'IForest'
clf = IForest(n_jobs=10)

In [ ]:
results = []

for filename in tqdm(preprocessed_dict.keys(), desc='Processing Time Series'):
    ts = preprocessed_dict[filename]
    x = ts['X_data']
    
    score = []
    for batch in tqdm(ts['batched_X_data'], desc='Processing Batch'):
        clf.fit(batch)
        score.extend(clf.decision_scores_)
    
    score = np.array(score)
    score = MinMaxScaler(feature_range=(0,1)).fit_transform(score.reshape(-1,1)).ravel()
    score = np.array([score[0]]*math.ceil((ts['slidingWindow']-1)/2) + list(score) + [score[-1]]*((ts['slidingWindow']-1)//2))
    
    L = printResult(ts['data'], ts['label'], score, ts['slidingWindow'], ts['name'], modelName)
    results.append([filename] + L)

In [ ]:
# columns = ['Filename'] + [f'Metric_{i+1}' for i in range(len(results[0])-1)]
columns = ['Filename'] + ['AUC', 'Precision', 'Recall', 'F-score', 'Range-recall', 'ExistenceReward', 'OverlapReward', 'Range-precision', 'Range-Fscore', 'Precison@k', 'RangeAUC']
df = pd.DataFrame(results, columns=columns)

In [ ]:
df

### ***STUMP***

In [ ]:
import stumpy
modelName = 'STUMP'

In [ ]:
results = []

for filename in tqdm(preprocessed_dict.keys(), desc='Processing Time Series'):
    ts = preprocessed_dict[filename]
    full_ts = ts['data']
    subseries = ts['X_data']
    window_size = ts['slidingWindow']
    slidingWindow = window_size

    k = 1
    score = []
    for batch in tqdm(ts['batched_data'], desc='Processing Batch'):
        score_ = stumpy.stump(T_A=batch, m=window_size, k=k, ignore_trivial=True, normalize=True)
    
        score.extend(score_.T[k-1])
        
    score = np.array(score)
    score = MinMaxScaler(feature_range=(0,1)).fit_transform(score.reshape(-1,1)).ravel()
    score = np.array([score[0]]*math.ceil((slidingWindow-1)/2) + list(score) + [score[-1]]*((slidingWindow-1)//2))
    
    L = printResult(ts['data'], ts['label'], score, ts['slidingWindow'], ts['name'], modelName)
    results.append([filename] + L)

In [ ]:
len(batch)

In [ ]:
columns = ['Filename'] + ['AUC', 'Precision', 'Recall', 'F-score', 'Range-recall', 'ExistenceReward', 'OverlapReward', 'Range-precision', 'Range-Fscore', 'Precison@k', 'RangeAUC']
df = pd.DataFrame(results, columns=columns)
df